In [21]:
import numpy as np
import pandas as pd
from selenium import webdriver
import gspread
from google.oauth2 import service_account
import json
import google.auth
from google.auth.transport.requests import AuthorizedSession
from google.oauth2.credentials import Credentials
from oauth2client.service_account import ServiceAccountCredentials
import re




#
#
# RUN SELENIUM

In [2]:
driver = webdriver.Chrome(r"/Users/jarkrunglerdkriangkrai/Github/setscraping/chromedriver")

#
#
# GET DATA

In [3]:
driver.get('http://siamchart.com/stock/')

In [4]:
data = driver.page_source

#
#
# CONVERT TO DATAFRAME

In [5]:
pd.read_html(data)[2]

,Name,No.,Links,Sign,Last,Chg%,Volume,Value (k),MCap (M),P/E,...,ROA%,ROE%,NPM%,Yield%,FFloat%,MG%,Magic1,Magic2,PEG,CG
0,Average,NaN,NaN,NaN,19.73,1.25,5377584.93,64616.07,25666.19,36.23,...,5.48,7.09,12.06,0.09,41.39,NaN,617.88,615.33,1.96,NaN
1,24CS,1.0,i,NaN,2.88,1.41,779800.00,2259.00,0.00,NaN,...,NaN,NaN,NaN,-,30.23,NaN,NaN,NaN,NaN,NaN
2,2S,2.0,i | 1 | 2 | 3,NaN,3.24,-1.22,221300.00,718.00,1793.00,NaN,...,-2.64,-4.19,-2.68,13.19,57.50,NaN,NaN,NaN,NaN,NaN
3,3K-BAT,3.0,i | 1 | 3,NaN,62.00,0.00,0.00,0.00,4861.00,29.33,...,6.46,7.05,3.05,0.50,2.10,NaN,649.00,619.00,0.29,NaN
4,7UP,4.0,i | 1 | 2 | 3,NaN,0.78,0.00,8881400.00,6977.00,4063.00,119.96,...,0.31,1.32,-12.96,NaN,63.37,NaN,1207.00,1262.00,-4.90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,YGG,822.0,i | 1 | 3,NaN,8.15,2.52,717700.00,5761.00,4876.00,40.08,...,23.60,24.66,36.59,0.86,45.15,NaN,567.00,517.00,0.49,NaN
823,YONG,823.0,i,NaN,2.20,1.85,3278900.00,7218.00,1523.00,17.20,...,9.49,11.42,9.25,NaN,27.24,NaN,538.00,488.00,NaN,NaN
824,YUASA,824.0,i | 1 | 3,NaN,15.20,0.00,445600.00,6854.00,1636.00,14.95,...,8.95,11.03,4.15,2.28,15.27,NaN,518.00,476.00,-24.83,NaN
825,ZEN,825.0,i | 1 | 3,NaN,17.30,1.17,379700.00,6474.00,5190.00,47.83,...,7.31,9.29,4.02,NaN,27.34,NaN,856.00,808.00,-2.77,NaN


In [6]:
df = pd.read_html(data)[2].drop(['No.','Links','CG','Sign','MG%'],axis=1).tail(-1)
df

,Name,Last,Chg%,Volume,Value (k),MCap (M),P/E,P/BV,D/E,DPS,EPS,ROA%,ROE%,NPM%,Yield%,FFloat%,Magic1,Magic2,PEG
1,24CS,2.88,1.41,779800.0,2259.0,0.0,NaN,-,NaN,-,0.00,NaN,NaN,NaN,-,30.23,NaN,NaN,NaN
2,2S,3.24,-1.22,221300.0,718.0,1793.0,NaN,0.90,0.26,0.27,0.00,-2.64,-4.19,-2.68,13.19,57.50,NaN,NaN,NaN
3,3K-BAT,62.00,0.00,0.0,0.0,4861.0,29.33,2.04,0.61,0.06,0.54,6.46,7.05,3.05,0.50,2.10,649.0,619.0,0.29
4,7UP,0.78,0.00,8881400.0,6977.0,4063.0,119.96,1.61,0.38,NaN,0.01,0.31,1.32,-12.96,NaN,63.37,1207.0,1262.0,-4.90
5,A,5.00,-0.99,1900.0,10.0,4880.0,NaN,1.54,3.42,NaN,0.00,2.11,-2.60,2.87,NaN,24.92,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,YGG,8.15,2.52,717700.0,5761.0,4876.0,40.08,8.02,0.05,0.08,0.20,23.60,24.66,36.59,0.86,45.15,567.0,517.0,0.49
823,YONG,2.20,1.85,3278900.0,7218.0,1523.0,17.20,1.90,0.63,NaN,0.13,9.49,11.42,9.25,NaN,27.24,538.0,488.0,NaN
824,YUASA,15.20,0.00,445600.0,6854.0,1636.0,14.95,1.60,0.59,0.35,1.02,8.95,11.03,4.15,2.28,15.27,518.0,476.0,-24.83
825,ZEN,17.30,1.17,379700.0,6474.0,5190.0,47.83,4.24,1.10,NaN,0.36,7.31,9.29,4.02,NaN,27.34,856.0,808.0,-2.77


In [7]:
import pickle
file_name='my_file.pkl'
f = open(file_name,'wb')
f.close()

In [8]:
df.to_pickle('/Users/jarkrunglerdkriangkrai/Github/setscraping/my_file.pkl')


#
#
# UPDATE TO GOOGLE SHEETS

In [9]:
import gspread
gc = gspread.service_account(filename='/Users/jarkrunglerdkriangkrai/Github/setscraping/service_account.json')

sh = gc.open("set scraping")


In [10]:
wks = sh.worksheet("Sheet1")

In [11]:
print('Rows: ', wks.row_count)
print('Columns: ', wks.col_count)

Rows:  1000
Columns:  26


In [12]:
#creds, project = google.auth.default(scopes=['https://www.googleapis.com/auth/spreadsheets'])
#service = build('sheets', 'v4', credentials=creds)

In [13]:
wks.clear()

{'spreadsheetId': '1sDLJJubzeMhc_U4xUe-P3rvSF7t7pV8tRKeabtux6WQ',
 'clearedRange': 'Sheet1!A1:Z1000'}

In [14]:
#wks.update('A1',df)

TypeError: Object of type DataFrame is not JSON serializable

In [ ]:
wks.update('A1',df)

{'spreadsheetId': '1sDLJJubzeMhc_U4xUe-P3rvSF7t7pV8tRKeabtux6WQ',
 'updatedRange': 'Sheet1!A1'}

In [4]:
driver.get('https://www.set.or.th/th/market/get-quote/stock/')

data_dim = driver.page_source


In [7]:
pd.read_html(data_dim)[1]


,ชื่อย่อหลักทรัพย์ (Click to sort Descending),ชื่อเต็มหลักทรัพย์จดทะเบียน (Click to sort Ascending),ตลาด (Click to sort Ascending),กลุ่มอุตสาหกรรม (Click to sort Ascending),หมวดธุรกิจ (Click to sort Ascending),Factsheet (Click to clear sorting)
0,24CS,บริษัท ทเวนตี้ โฟร์ คอน แอนด์ ซัพพลาย จำกัด (ม...,mai,PROPCON,-,NaN
1,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,INDUS,STEEL,NaN
2,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,INDUS,AUTO,NaN
3,7UP,บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...,SET,RESOURC,ENERG,NaN
4,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,PROPCON,PROP,NaN
5,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,PROPCON,-,NaN
6,AAI,บริษัท เอเชี่ยน อะไลอันซ์ อินเตอร์เนชั่นแนล จำ...,SET,AGRO,FOOD,NaN
7,AAV,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน),SET,SERVICE,TRANS,NaN
8,ABM,บริษัท เอเชีย ไบโอแมส จำกัด (มหาชน),mai,RESOURC,-,NaN
9,ACAP,บริษัท เอเชีย แคปปิตอล กรุ๊ป จำกัด (มหาชน),mai,FINCIAL,-,NaN


In [9]:
driver = webdriver.Chrome(r'/Users/jarkrunglerdkriangkrai/Github/setscraping/chromedriver')
driver.get('http://siamchart.com/stock-financial/')
data = driver.page_source

In [20]:
df = pd.read_html(data)[0].drop('No.',axis=1)
df

,Name,x EPS17 (YoY%),x EPS18 (YoY%),x EPS19 (YoY%),x EPS20 (YoY%),x EPS21 (YoY%),x Rev17 (YoY%),x Rev18 (YoY%),x Rev19 (YoY%),x Rev20 (YoY%),x Rev21 (YoY%),x NP17 (YoY%),x NP18 (YoY%),x NP19 (YoY%),x NP20 (YoY%),x NP21 (YoY%),x PE
0,24CS,0 (NaN),0 (NaN),0 (NaN),0 (NaN),0 (NaN),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2S,0.71 (-37.72),0.23 (-67.61),0.30 (+30.43),0.91 (+203.33),1.47 (+61.54),"4,535.20 (+14.91)","5,462.98 (+20.46)","5,413.94 (-0.90)","5,625.92 (+3.92)","6,555.13 (+16.52)",284.71 (-17.01),105.55 (-62.93),133.38 (+26.37),410.17 (+207.52),737.52 (+79.81),NaN
2,3K-BAT,0 (NaN),0 (NaN),0 (NaN),0.54 (+Infinity),1.96 (+262.96),NaN,NaN,5707.58,"4,782.61 (-16.21)","5,420.57 (+13.34)",NaN,NaN,109.85,42.49 (-61.32),153.50 (+261.26),29.58
3,7UP,0 (NaN),0 (NaN),0.03 (+Infinity),0.04 (+33.33),0.06 (+50.00),626.35,791.20 (+26.32),"1,142.07 (+44.35)","1,365.02 (+19.52)",799.54 (-41.43),-211.19,-79.53 (+62.34),86.61 (+208.90),117.44 (+35.60),280.58 (+138.91),119.96
4,A,0.10 (-62.96),0.03 (-70.00),0 (-100.00),0.2 (+Infinity),0 (-100.00),"5,057.09 (+7.03)","4,038.88 (-20.13)","2,988.57 (-26.00)","5,740.86 (+92.09)","3,283.20 (-42.81)",97.69 (-62.90),30.79 (-68.48),-280.61 (-1011.37),194.87 (+169.45),-345.56 (-277.33),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,YGG,0 (NaN),0 (NaN),0 (NaN),0.31 (+Infinity),0.62 (+100.00),NaN,173.54,196.60 (+13.29),228.06 (+16.00),287.70 (+26.15),NaN,21.09,48.63 (+130.58),56.57 (+16.33),112.05 (+98.07),39.59
824,YONG,0 (NaN),0 (NaN),0 (NaN),0 (NaN),0 (NaN),NaN,NaN,NaN,NaN,881.42,NaN,NaN,NaN,NaN,68.94,16.74
825,YUASA,1.04 (-1.89),1.19 (+14.42),1.29 (+8.40),1.47 (+13.95),0.91 (-38.10),"2,338.26 (+11.90)","2,404.49 (+2.83)","2,257.79 (-6.10)","2,075.55 (-8.07)","2,261.67 (+8.97)",111.46 (-2.00),128.20 (+15.02),138.92 (+8.36),158.18 (+13.86),97.68 (-38.25),14.75
826,ZEN,0 (NaN),0 (NaN),0.36 (+Infinity),-0.21 (-158.33),0 (+100.00),2515.17,"2,964.65 (+17.87)","3,144.17 (+6.06)","2,333.39 (-25.79)","2,255.32 (-3.35)",81.81,140.21 (+71.38),106.22 (-24.24),-63.66 (-159.93),-91.57 (-43.84),47.56


In [28]:
df.columns

Index(['Name', 'x EPS17 (YoY%)', 'x EPS18 (YoY%)', 'x EPS19 (YoY%)',
       'x EPS20 (YoY%)', 'x EPS21 (YoY%)', 'x Rev17 (YoY%)', 'x Rev18 (YoY%)',
       'x Rev19 (YoY%)', 'x Rev20 (YoY%)', 'x Rev21 (YoY%)', 'x NP17 (YoY%)',
       'x NP18 (YoY%)', 'x NP19 (YoY%)', 'x NP20 (YoY%)', 'x NP21 (YoY%)',
       'x PE'],
      dtype='object')

In [27]:
import pandas as pd



# replace the parentheses and the values inside with an empty string
df['x EPS17 (YoY%)'] = df['x EPS17 (YoY%)'].str.replace(r'\s*\([^\)]*\)\s*', '', regex=True)

print(df)


       Name x EPS17 (YoY%) x EPS18 (YoY%)    x EPS19 (YoY%)    x EPS20 (YoY%)  \
0      24CS              0        0 (NaN)           0 (NaN)           0 (NaN)   
1        2S           0.71  0.23 (-67.61)     0.30 (+30.43)    0.91 (+203.33)   
2    3K-BAT              0        0 (NaN)           0 (NaN)  0.54 (+Infinity)   
3       7UP              0        0 (NaN)  0.03 (+Infinity)     0.04 (+33.33)   
4         A           0.10  0.03 (-70.00)       0 (-100.00)   0.2 (+Infinity)   
..      ...            ...            ...               ...               ...   
823     YGG              0        0 (NaN)           0 (NaN)  0.31 (+Infinity)   
824    YONG              0        0 (NaN)           0 (NaN)           0 (NaN)   
825   YUASA           1.04  1.19 (+14.42)      1.29 (+8.40)     1.47 (+13.95)   
826     ZEN              0        0 (NaN)  0.36 (+Infinity)   -0.21 (-158.33)   
827    ZIGA           0.28  0.08 (-71.43)     0.07 (-12.50)    0.23 (+228.57)   

     x EPS21 (YoY%)     x R

In [29]:
# loop through all columns in the dataframe
for col in df.columns:
    # replace values in column if it's a string type
    if df[col].dtype == 'O':
        df[col] = df[col].str.replace(r'\s*\([^\)]*\)\s*', '', regex=True)


In [30]:
print(df)

       Name x EPS17 (YoY%) x EPS18 (YoY%) x EPS19 (YoY%) x EPS20 (YoY%)  \
0      24CS              0              0              0              0   
1        2S           0.71           0.23           0.30           0.91   
2    3K-BAT              0              0              0           0.54   
3       7UP              0              0           0.03           0.04   
4         A           0.10           0.03              0            0.2   
..      ...            ...            ...            ...            ...   
823     YGG              0              0              0           0.31   
824    YONG              0              0              0              0   
825   YUASA           1.04           1.19           1.29           1.47   
826     ZEN              0              0           0.36          -0.21   
827    ZIGA           0.28           0.08           0.07           0.23   

    x EPS21 (YoY%) x Rev17 (YoY%) x Rev18 (YoY%) x Rev19 (YoY%)  \
0                0            Na